<a href="https://colab.research.google.com/github/dhrits/mec2-projects/blob/main/DSagar_Student_MLE_MiniProject_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Transfer Learning with Keras

Transfer learning is a machine learning technique where a model trained on one task is used as a starting point to solve a different but related task. Instead of training a model from scratch, transfer learning leverages the knowledge learned from the source task and applies it to the target task. This approach is especially useful when the target task has limited data or computational resources.

In transfer learning, the pre-trained model, also known as the "base model" or "source model," is typically trained on a large dataset and a more general problem (e.g., image classification on ImageNet, a vast dataset with millions of labeled images). The knowledge learned by the base model in the form of feature representations and weights captures common patterns and features in the data.

To perform transfer learning, the following steps are commonly followed:

1. Pre-training: The base model is trained on a source task using a large dataset, which can take a considerable amount of time and computational resources.

2. Feature Extraction: After pre-training, the base model is used as a feature extractor. The last few layers (classifier layers) of the model are discarded, and the remaining layers (feature extraction layers) are retained. These layers serve as feature extractors, producing meaningful representations of the data.

3. Fine-tuning: The feature extraction layers and sometimes some of the earlier layers are connected to a new set of layers, often called the "classifier layers" or "task-specific layers." These layers are randomly initialized, and the model is trained on the target task with a smaller dataset. The weights of the base model can be frozen during fine-tuning, or they can be allowed to be updated with a lower learning rate to fine-tune the model for the target task.

Transfer learning has several benefits:

1. Reduced training time and resource requirements: Since the base model has already learned generic features, transfer learning can save time and resources compared to training a model from scratch.

2. Improved generalization: Transfer learning helps the model generalize better to the target task, especially when the target dataset is small and dissimilar from the source dataset.

3. Better performance: By starting from a model that is already trained on a large dataset, transfer learning can lead to better performance on the target task, especially in scenarios with limited data.

4. Effective feature extraction: The feature extraction layers of the pre-trained model can serve as powerful feature extractors for different tasks, even when the task domains differ.

Transfer learning is commonly used in various domains, including computer vision, natural language processing (NLP), and speech recognition, where pre-trained models are fine-tuned for specific applications like object detection, sentiment analysis, or speech-to-text.

In this mini-project you will perform fine-tuning using Keras with a pre-trained VGG16 model on the CIFAR-10 dataset.

First, import all the libraries you'll need.

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

The CIFAR-10 dataset is a widely used benchmark dataset in the field of computer vision and machine learning. It stands for the "Canadian Institute for Advanced Research 10" dataset. CIFAR-10 was created by researchers at the CIFAR institute and was originally introduced as part of the Neural Information Processing Systems (NIPS) 2009 competition.

The dataset consists of 60,000 color images, each of size 32x32 pixels, belonging to ten different classes. Each class contains 6,000 images. The ten classes in CIFAR-10 are:

1. Airplane
2. Automobile
3. Bird
4. Cat
5. Deer
6. Dog
7. Frog
8. Horse
9. Ship
10. Truck

The images are evenly distributed across the classes, making CIFAR-10 a balanced dataset. The dataset is divided into two sets: a training set and a test set. The training set contains 50,000 images, while the test set contains the remaining 10,000 images.

CIFAR-10 is often used for tasks such as image classification, object recognition, and transfer learning experiments. The relatively small size of the images and the variety of classes make it a challenging dataset for training machine learning models, especially deep neural networks. It also serves as a good dataset for teaching and learning purposes due to its manageable size and straightforward class labels.

Here are your tasks:

1. Load the CIFAR-10 dataset after referencing the documentation [here](https://keras.io/api/datasets/cifar10/).
2. Normalize the pixel values so they're all in the range [0, 1].
3. Apply One Hot Encoding to the train and test labels using the [to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function.
4. Further split the the training data into training and validation sets using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Use only 10% of the data for validation.  

In [29]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(f'shape of train data: X: {x_train.shape}, y: {y_train.shape}')
print(f'shape of test data: X: {x_test.shape}, y: {y_test.shape}')

shape of train data: X: (50000, 32, 32, 3), y: (50000, 1)
shape of test data: X: (10000, 32, 32, 3), y: (10000, 1)


In [30]:
# Normalize the pixel values to [0, 1]
def standardize(x, min=0., max=1.):
  x = (x - x.min()) / (x.max() - x.min())
  x = x * (max - min) + min
  return x

x_train = standardize(x_train)
x_test = standardize(x_test)
print(f'min, max values in training pixels {x_train.min()}, {x_train.max()}')
print(f'min, max values in testing pixels {x_test.min()}, {x_test.max()}')

min, max values in training pixels 0.0, 1.0
min, max values in testing pixels 0.0, 1.0


In [31]:
# One-hot encode the labels
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

assert y_train.shape[-1] == 10
assert y_test.shape[-1] == 10

In [32]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.02, random_state=0)

VGG16 (Visual Geometry Group 16) is a deep convolutional neural network architecture that was developed by the Visual Geometry Group at the University of Oxford. It was proposed by researchers Karen Simonyan and Andrew Zisserman in their paper titled "Very Deep Convolutional Networks for Large-Scale Image Recognition," which was presented at the International Conference on Learning Representations (ICLR) in 2015.

The VGG16 architecture gained significant popularity for its simplicity and effectiveness in image classification tasks. It was one of the pioneering models that demonstrated the power of deeper neural networks for visual recognition tasks.

Key characteristics of the VGG16 architecture:

1. Architecture: VGG16 consists of a total of 16 layers, hence the name "16." These layers are stacked one after another, forming a deep neural network.

2. Convolutional Layers: The main building blocks of VGG16 are the convolutional layers. It primarily uses 3x3 convolutional filters throughout the network, which allows it to capture local features effectively.

3. Max Pooling: After each set of convolutional layers, VGG16 applies max-pooling layers with 2x2 filters and stride 2, which halves the spatial dimensions (width and height) of the feature maps and reduces the number of parameters.

4. Fully Connected Layers: Towards the end of the network, VGG16 has fully connected layers that act as a classifier to make predictions based on the learned features.

5. Activation Function: The network uses the Rectified Linear Unit (ReLU) activation function for all hidden layers, which helps with faster convergence during training.

6. Number of Filters: The number of filters in each convolutional layer is relatively small compared to more recent architectures like ResNet or InceptionNet. However, stacking multiple layers allows VGG16 to learn complex hierarchical features.

7. Output Layer: The output layer consists of 1000 units, corresponding to 1000 ImageNet classes. VGG16 was originally trained on the large-scale ImageNet dataset, which contains millions of images from 1000 different classes.

VGG16 was instrumental in showing that increasing the depth of a neural network can significantly improve its performance on image recognition tasks. However, the main drawback of VGG16 is its high number of parameters, making it computationally expensive and memory-intensive to train. Despite this limitation, VGG16 remains an essential benchmark architecture and has paved the way for even deeper and more efficient models in the field of computer vision, such as ResNet, DenseNet, and EfficientNet.

Here are your tasks:

1. Load [VGG16](https://keras.io/api/applications/vgg/#vgg16-function) as a base model. Make sure to exclude the top layer.
2. Freeze all the layers in the base model. We'll be using these weights as a feature extraction layer to forward to layers that are trainable.

In [33]:
# Load the pre-trained VGG16 model (excluding the top classifier)
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

model = VGG16(include_top=False, weights='imagenet')

In [34]:
# Freeze the layers in the base model
for l in model.layers:
  l.trainable = False

Now, we'll add some trainable layers to the base model.

1. Using the base model, add a [GlobalAveragePooling2D](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) layer, followed by a [Dense](https://keras.io/api/layers/core_layers/dense/) layer of length 256 with ReLU activation. Finally, add a classification layer with 10 units, corresponding to the 10 CIFAR-10 classes, with softmax activation.
2. Create a Keras [Model](https://keras.io/api/models/model/) that takes in approproate inputs and outputs.

In [35]:
# Add a global average pooling layer
from tensorflow.keras.layers import GlobalAvgPool2D, Dense
gap = GlobalAveragePooling2D()(model.layers[-1].output)

In [36]:
# Add a fully connected layer with 256 units and ReLU activation
dense = Dense(256, activation='relu')(gap)

In [37]:
# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation
classification = Dense(10, activation='softmax')(dense)

In [38]:
# Create the fine-tuned model
from tensorflow.keras.models import Model
cifar_model = Model(model.input, classification)
cifar_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

With your model complete it's time to train it and assess its performance.

1. Compile your model using an appropriate loss function. Feel free to play around with the optimizer, but a good starting optimizer might be Adam with a learning rate of 0.001.
2. Fit your model on the training data. Use the validation data to print the accuracy for each epoch. Try training for 10 epochs. Note, training can take a few hours so go ahead and grab a cup of coffee.

**Optional**: See if you can implement an [Early Stopping](https://keras.io/api/callbacks/early_stopping/) criteria as a callback function.

In [39]:
# Compile the model
from tensorflow.keras.optimizers import Adam
cifar_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

history = cifar_model.fit(x_train, y_train, batch_size=512,
                          epochs=10,
                          validation_data=(x_val, y_val),
                          callbacks=[early_stopping])

Epoch 1/10
96/96 [==============================] - 5s 46ms/step - loss: 1.5627 - accuracy: 0.4636 - val_loss: 1.3408 - val_accuracy: 0.5370
Epoch 2/10
96/96 [==============================] - 2s 21ms/step - loss: 1.2816 - accuracy: 0.5579 - val_loss: 1.2555 - val_accuracy: 0.5490
Epoch 3/10
96/96 [==============================] - 2s 22ms/step - loss: 1.2097 - accuracy: 0.5818 - val_loss: 1.2066 - val_accuracy: 0.5800
Epoch 4/10
96/96 [==============================] - 2s 23ms/step - loss: 1.1691 - accuracy: 0.5963 - val_loss: 1.1875 - val_accuracy: 0.5910
Epoch 5/10
96/96 [==============================] - 2s 21ms/step - loss: 1.1381 - accuracy: 0.6054 - val_loss: 1.1646 - val_accuracy: 0.5810
Epoch 6/10
96/96 [==============================] - 2s 21ms/step - loss: 1.1112 - accuracy: 0.6130 - val_loss: 1.1518 - val_accuracy: 0.6070
Epoch 7/10
96/96 [==============================] - 2s 23ms/step - loss: 1.0936 - accuracy: 0.6195 - val_loss: 1.1612 - val_accuracy: 0.5880
Epoch 8/10
96

With your model trained, it's time to assess how well it performs on the test data.

1. Use your trained model to calculate the accuracy on the test set. Is the model performance better than random?
2. Experiment! See if you can tweak your model to improve performance.  

In [41]:
# Evaluate the model on the test set
cifar_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 6ms/step - loss: 1.1278 - accuracy: 0.6045


[1.1278353929519653, 0.6044999957084656]

Seems the model achieves an accuracy of roughly 60% on test data. **Lets try more epochs to allow the early stopping to kick in**

In [50]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import GlobalAvgPool2D, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import L2

def build_model(learning_rate=0.001, kernel_regularizer=None, freeze_until=None,
                hidden_layers=(256,), norm=False):
  model = VGG16(include_top=False, weights='imagenet')
  for l in model.layers[:freeze_until]:
    l.trainable = False
  gap = GlobalAveragePooling2D()(model.layers[-1].output)
  inp = gap
  for l in hidden_layers:
    inp = Dense(l, activation='relu', kernel_regularizer=kernel_regularizer)(inp)
    if norm:
      inp = BatchNormalization()(inp)
  classification = Dense(10, activation='softmax', kernel_regularizer=kernel_regularizer)(inp)
  cifar_model = Model(model.input, classification)
  cifar_model.summary()
  cifar_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
  return cifar_model

In [43]:
cifar_model = build_model()
history = cifar_model.fit(x_train, y_train, batch_size=1024,
                          epochs=100,
                          validation_data=(x_val, y_val),
                          callbacks=[early_stopping])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

In [44]:
cifar_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.1139 - accuracy: 0.6165


[1.113943099975586, 0.6165000200271606]

The above model does better  but starts to overfit as can be seen in the output. Lets add regularization and increase learning rate.


In [51]:
cifar_model = build_model(learning_rate=0.01, freeze_until=-3)
history = cifar_model.fit(x_train, y_train, batch_size=32,
                          epochs=200,
                          validation_data=(x_val, y_val),
                          callbacks=EarlyStopping(patience=15, monitor='val_accuracy'))

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

In [52]:
cifar_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.1526 - accuracy: 0.6263


[1.1525980234146118, 0.6262999773025513]

Now lets try freezing fewer layers and a smaller batch-size for more gradient updates.

In [53]:
cifar_model = build_model(learning_rate=0.001, freeze_until=-4)
history = cifar_model.fit(x_train, y_train, batch_size=64,
                          epochs=200,
                          validation_data=(x_val, y_val),
                          callbacks=EarlyStopping(patience=15, monitor='val_accuracy'))

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0  

In [57]:
cifar_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.5739 - accuracy: 0.7402


[1.573912501335144, 0.7401999831199646]

In [55]:
cifar_model.save('best_cifar.keras')

So freezing fewer layers seems to boost accuracy on validation and test sets significantly.